In [ ]:
import torch, os, sys, h5py, time
import numpy as np
from datetime import datetime
#dedicate packages import
sys.path.append(r'C:\Users\hvanvoorst\PhD\git_repos\FU2BL-GAN')
from options.options_FU2BL import get_options
from utils.TrainUtils import get_scheduler
from scripts.Train_FU2BL import FU2BL_GAN
from scripts.init_train import init_loader
#from utils.Augment import * # import all augmentation functions
from utils.Utils import *

opt = get_options()
# set path to location to store checkpoints and h5py file for loading data
opt.loc_checkpoints = r''
opt.train_filepath = r''
print(opt.train_filepath, opt.input_n_slices, opt.cache_size, opt.loc_checkpoints)

#import the loader for training
train_ldr, train_dataset, TSS = init_loader(opt)

In [ ]:
# initialize the model class (includes Discriminator and Generator models)
model = FU2BL_GAN(opt)
print(model.loc_checkpoints)

if not os.path.exists(model.loc_checkpoints):
    os.makedirs(model.loc_checkpoints)

eps = opt.n_epochs_same+opt.n_epochs_decay+2
print('Start training at:', datetime.now(), 
      '\nStarting with epoch number:', model.epoch, 
      '\nTraining for', eps, 'epochs')

# initialize start epoch and lr scheduling 
opt.epoch_count = model.epoch
scheduler_D = get_scheduler(model.optimizer_D,opt)
scheduler_G = get_scheduler(model.optimizer_G,opt)

total_iters = model.iter
train_G, train_D = True, True #allows for alternated training (more D than G for example)
for epoch in range(model.epoch,eps):
    epoch_start = time.time()
    for i,(BL,FU,ID) in enumerate(train_ldr): 
        t1 = time.time()
        total_iters += BL.shape[0]

        model.set_input(FU, BL) 
        model.optimize_parameters()

        t2 = time.time()
        t = round(t2-t1)
        model.at_iter_end(epoch,total_iters,ID, train_G,train_D,t, 
                          save=True, val_loader=None, # set val_loader to None if no validation set
                          epoch_end=(i==len(train_ldr)-1))


    print('Epoch:',epoch,'--iteration:', i, 
          '--total-n-slices:',total_iters,
          train_G, train_D,#multistop_D,
          'G-loss:', round(model.loss_G.item(),3), 
          'D-loss:',round(model.loss_D.item(),3),'\n',
          'Fake acc:', round(model.acc_fake,2),
          'Real acc:', round(model.acc_real,2),
          t,len(model.errors))

    scheduler_D.step()
    scheduler_G.step()

    epoch_end = time.time()

    print('finished epoch:', epoch, 'in:', round(epoch_end-epoch_start), 'seconds')
print('Finished training:',datetime.now())